In [ ]:
# Jupyter나 VS Code에서 실행 시 사용할 수 있도록 수정한 버전

import os
from openai import AzureOpenAI
from dotenv import load_dotenv

def process_file(input_file_path, output_file_path=None):
    """
    Process an input file through Azure AI to convert HTML tables to text.
    """
    load_dotenv()
    endpoint = os.getenv("ENDPOINT_URL")
    deployment = os.getenv("DEPLOYMENT_NAME")
    subscription_key = os.getenv("AZURE_OPENAI_KEY")
    
    if not all([endpoint, deployment, subscription_key]):
        raise ValueError("환경변수 누락: ENDPOINT_URL, DEPLOYMENT_NAME, AZURE_OPENAI_KEY를 확인하세요.")
    
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=subscription_key,
        api_version="2024-05-01-preview",
    )
    
    with open(input_file_path, 'r', encoding='utf-8') as file:
        input_content = file.read()
    
    chat_prompt = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "너는 HTML 테이블을 읽고 자연스러운 서술형 텍스트로 변환하는 텍스트 변환 엔진이다.\n\n입력 데이터는 일반 텍스트와 HTML 코드가 혼합된 문서이며, 이 중 HTML 테이블(`<table>`) 형식으로 작성된 표만을 감지하여 사람이 읽기 쉬운 **자연스러운 텍스트**로 변환하라. 표 외의 일반 텍스트는 **절대로 변경하지 않는다**. \n\n출력된 텍스트는 아래의 기준을 모두 따라야 한다:\n\n1. 표의 계층 구조, 제목, 셀의 관계를 모두 파악하여 자연어로 기술한다.\n2. 셀이 병합된 경우 (`rowspan`, `colspan`)에는 의미적으로 내용을 통합하여 풀어서 설명한다.\n3. 표 안에 또 다른 표가 중첩되어 있는 경우에도 각 표를 계층적으로 처리하고, 문맥상 자연스럽게 연결되도록 한다.\n4. 빈 칸이 있는 경우, 내용을 유추하지 않고 \"(빈칸)\" 또는 \"해당 없음\" 등으로 명확하게 표기한다.\n5. 항목 간 구분은 \"■\", \"1.\", \"-\" 등을 사용하여 명확히 구분하고, 계층적으로 정리한다.\n6. 결과 텍스트는 반드시 문맥상 자연스럽고 일관되게 연결되어야 하며, 원래 문서의 흐름과 연결되도록 이어져야 한다.\n7. HTML 태그가 아닌 일반 텍스트 영역은 절대로 수정하거나 재구성하지 않는다.\n8. 결과는 마크다운 문서로 사용 가능한 수준의 가독성을 갖춰야 하며, 표를 설명하는 문장은 공식 문서나 계약서 스타일처럼 명료하고 단정하게 작성한다.\n\n예외나 애매한 구조가 있어도 최대한 의미를 보존하여 사람이 이해할 수 있도록 직관적으로 설명하라.\n\n입력 형식 예시:\n(본문 텍스트)\n<table>...</table>\n(본문 텍스트 계속)\n\n출력 형식 예시:\n(본문 텍스트)\n■ 항목명  \n- 내용1  \n- 내용2  \n이제 아래에 입력된 문서 내 HTML 테이블을 위 기준에 따라 서술형 텍스트로 변환하라."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": input_content
                }
            ]
        }
    ]
    
    completion = client.chat.completions.create(
        model=deployment,
        messages=chat_prompt,
        max_tokens=1500,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stream=False
    )
    
    processed_text = completion.choices[0].message.content
    
    if not output_file_path:
        base, _ = os.path.splitext(input_file_path)
        output_file_path = f"{base}_processed.txt"
    
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(processed_text)
    
    print(f"✅ 변환 완료: {output_file_path}")
    return processed_text


# 🔽 여기서 input/output 경로 지정해서 실행
input_path = "document_result.md"  # ← 실제 파일 경로로 수정
output_path = "result.txt"  # 또는 "결과저장파일.txt"

process_file(input_path, output_path)


✅ 변환 완료: result.txt


'(본문 텍스트)\n\n### 모집 일정에 대한 설명\n\n■ 모집 일정은 다음과 같이 진행됩니다:  \n1. **모집 공고**  \n   - 2025년 3월 27일(목)에 공고됩니다.  \n2. **신청 접수**  \n   - 인터넷 및 모바일을 통해 2025년 4월 7일(월)부터 4월 9일(수)까지 접수 가능합니다.  \n3. **서류심사 대상자 발표**  \n   - 2025년 4월 11일(금)에 발표됩니다.  \n4. **서류 제출**  \n   - 선정된 대상자는 2025년 4월 14일(월)부터 4월 16일(수)까지 서류를 등기우편으로 제출해야 합니다.  \n5. **자격 검증**  \n   - 자격 검증은 개별 안내를 통해 진행됩니다.  \n6. **예비입주자 순번 발표**  \n   - 2025년 6월 13일(금)에 발표됩니다.  \n7. **계약 체결**  \n   - 예비입주자 순번 도래 시 계약이 체결됩니다.  \n8. **입주**  \n   - 계약일부터 60일 이내에 입주를 완료해야 합니다.  \n\n---\n\n### 임대기간 및 임대조건\n\n■ 임대기간  \n- 최초 임대기간은 2년이며, 자격 충족 시 2년 단위로 최대 4회 재계약 가능(최장 10년 거주).  \n- 입주 후 혼인한 경우, 재계약 횟수 5회 추가 가능(최장 20년 거주).  \n\n■ 임대조건  \n- 시중 시세의 40~50% 수준으로 임대료 및 보증금이 책정됩니다.  \n- 1순위는 시중 시세의 40%(임대보증금 100만 원), 2·3순위는 시중 시세의 50%(임대보증금 200만 원)로 적용됩니다.  \n- 임대보증금을 증액하여 월임대료를 낮출 수 있으며, 전환한도는 기본 월임대료의 최대 60%까지 가능합니다. 예를 들어, 300만 원 추가 납부 시 월임대료가 약 17,500원 감소합니다.  \n\n---\n\n### 입주자격 및 입주순위\n\n■ 입주자격  \n- 공고일 기준(2025년 3월 27일) 무주택자인 미혼 청년(19세 이상 39세 이하)으로 소득 및 자

: 